# Tugas 1 (Crawling Data)

*   Pendapat Masyarakat tentang Ganjar Pranowo yang maju sebagai calon presiden periode 2024 - 2029 (Twitter)
*   Portal Tugas Akhir Universitas Trunojoyo
*   Berita Online

## Persiapan Environment

### Instalasi Library SNSCRAPE

In [1]:
!pip3 install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 KB 3.0 MB/s eta 0:00:00


### Instalasi BeautifulSoup

In [2]:
!pip install requests
!pip install html5lib
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=a0a2c84e23b0e847d6c9f584677a87ad3f8312b284539eb884d796125deb1a7a
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [3]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

### Pembuatan Fungsi untuk Analisis Sentimen

In [4]:
from textblob import TextBlob
import re
def clean_tweet(tweet):
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
  analysis = TextBlob(clean_tweet(tweet))
  if analysis.sentiment.polarity > 0:
    return 1
  elif analysis.sentiment.polarity == 0:
    return 0
  else:
    return -1

  def analize_subjectorobject(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
      return 'Positif'
    elif analysis.sentiment.polarity == 0:
      return 'Neutral'
    else:
      return 'Negatif'
    

## Crawling Data dari Twitter dan Konversi Hasil Data ke CSV

In [5]:
tweets = []

In [6]:
# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('Ganjar Pranowo Calon Presiden since:2022-01-01 until:2023-02-20').get_items()):
    if i>500:
        break
    tweets.append([tweet.date, tweet.user.username, tweet.content, analize_sentiment(tweet.content), tweet.user.location ])

# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets, columns=['Datetime', 'Username Twitter', 'Tweet', 'Label', 'Lokasi'])

<ipython-input-6-ef96f4d4f9f2>:5: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.user.username, tweet.content, analize_sentiment(tweet.content), tweet.user.location ])


In [7]:
tweets_df['Label'] = tweets_df['Label'].replace([0,1,-1], ['Netral', 'Positif', 'Negatif'])

In [8]:
tweets_df

,Datetime,Username Twitter,Tweet,Label,Lokasi
0,2023-02-19 13:45:59+00:00,GustiMberkahi,LAMPUNG PUNYA CERITA\n\nRelawan Ganjar Pranowo...,Netral,Yogya 9/9/2020
1,2023-02-19 11:08:25+00:00,__RismaWidiono_,"Sekitar 10 ribu petani di Temanggung, Jawa Ten...",Netral,"DKI Jakarta, Indonesia"
2,2023-02-19 03:11:00+00:00,nurmalianindi,Calon Presiden 2024 nih bos #ganjarpranowo #ga...,Netral,"Bengkulu, Indonesia"
3,2023-02-18 11:01:08+00:00,evalita26,@AirinDatangLagi @ganjarpranowo Yang waras ten...,Netral,
4,2023-02-18 11:00:10+00:00,SasliRahadi1,@ganjarpranowo Calon presiden tapi presiden yo...,Netral,
...,...,...,...,...,...
496,2023-01-12 08:01:43+00:00,gracia_tejo,@GanjaranApp @PDI_Perjuangan @ganjarpranowo ma...,Netral,"Blora, Indonesia"
497,2023-01-12 07:22:30+00:00,GanjaranApp,HUT @PDI_Perjuangan ke 50 tahun itu merupakan ...,Netral,
498,2023-01-12 06:47:01+00:00,Urrangawak,Sama kita Pak Guntur Soekarnoputra untuk calon...,Netral,🇮🇩 Pancasila
499,2023-01-12 05:38:53+00:00,IDSahabatGanjar,Dalam setiap kali survei soal calon presiden y...,Netral,Indonesia


In [9]:
tweets_df.to_csv('ganjar.csv', index=False)

## Crawling Data dari Portal Tugas Akhir Universitas Trunojoyo

In [10]:
import requests
from bs4 import BeautifulSoup

pta = []

for i in range(10,61):
  URL = "https://pta.trunojoyo.ac.id/c_search/byprod/10/" + str(i+1)
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  items = soup.find('div', {'id':'content_journal'})

  for it in items.findAll('li',{'data-cat':'#luxury'}):
    tempNama = it.span.text.strip('\n').replace('Penulis : ', '')
    # links = it.findAll('div', {'style':'margin: 25px 0px 15px 0px;'})
    link = it.find('a', 'gray button')['href']
    req = requests.get(link)
    soup2 = BeautifulSoup(req.text, 'html.parser')
    abstraks = soup2.findAll('div', {'style':'margin: 15px 15px 15px 15px;'})
    abstrak = soup2.find('p', {'align':'justify'}).text
  
    tempNIM = link.replace('https://pta.trunojoyo.ac.id/welcome/detail/', '')

    pta.append([it.a.text.strip('\n'), tempNIM, tempNama, abstrak])
    
pta_df = pd.DataFrame(pta, columns=['Judul', 'NIM', 'Penulis', 'Abstrak'])
pta_df

KeyboardInterrupt: ignored

In [ ]:
pta_df.to_csv('pta.csv', index=False)

## Crawling Data dari Portal Berita Online (Tempo)

In [ ]:
import requests
from bs4 import BeautifulSoup

berita = []

URL = "https://www.tempo.co/indeks/2023-02-21/"
r = requests.get(URL)
  
soup = BeautifulSoup(r.text, 'html.parser')
items = soup.find('main', 'main-left')

for it in items.findAll('h2','title'):
  judul = it.a.text
  link = it.a['href']

  req = requests.get(link)
    
  soup2 = BeautifulSoup(req.text, 'html.parser')
  contents = soup2.find('div', 'detail-in').text.strip('\nTEMPO.CO,')

  berita.append([judul, contents])
    
berita_df = pd.DataFrame(berita, columns=['Judul', 'Isi Berita'])
berita_df

In [ ]:
berita_df.to_csv('berita.csv', index=False)